In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier


In [2]:
torch.cuda.is_available()

True

In [3]:
xlf = DecisionTreeClassifier(random_state=0)

In [13]:
enr_vec = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/enroll/spkvecs.npy')
enr_len = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/enroll/veclabs.npy')


In [14]:
enr_cos_sim = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/test/result.npy')
not_cls_sim = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/notenrollled/result.npy')

enr_cls = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/test/answer.npy')
not_cls = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/notenrollled/answer.npy')+51

In [15]:
enr_cos_t = enr_vec.transpose()
print(enr_cos_t.shape)

avg_enr_vec = []
enr_lab = []
for i in range(1, len(enr_len)):
    avg_enr_vec.append(enr_cos_t[enr_len[i-1]:enr_len[i],:].mean(axis=0).reshape(1,-1))
    
    for x in range(enr_len[i]-enr_len[i-1]):
        
        enr_lab.append(i)


(1392, 128)


In [16]:
avg_center = np.concatenate(avg_enr_vec, axis=0).transpose()
avg_center =  torch.tensor(avg_center).float()
# avg_center = F.normalize(avg_center, dim=1) #* 12

print(avg_center.shape)
print(enr_cos_t.shape, enr_lab.__len__())

torch.Size([128, 50])
(1392, 128) 1392


In [17]:
print(not_cls_sim.shape)
enr_cos_t = torch.tensor(enr_cos_t).float()
# enr_cos_t = F.normalize(enr_cos_t, dim=1) #* 12

enr_cos = torch.matmul(enr_cos_t, avg_center)

# x = np.append(enr_cos, not_cls_sim[:100, :], axis=0)
# y = np.append(enr_lab, not_cls[:100])

x = enr_cos.numpy()
y = enr_lab

xlf.fit(x, y)

(11699, 50)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [18]:
# pre_x = np.append(enr_cos_sim, not_cls_sim[100:], axis=0)
# print(pre_x.shape)

# real_y = np.append(enr_cls, not_cls[100:])
# print(y.shape)

pre_x = enr_cos_sim
print(pre_x.shape)

real_y = np.array(enr_cls)
print(real_y.shape)


(31487, 50)
(31487,)


In [19]:
pred_y = xlf.predict(pre_x)

In [20]:
correct = 0
for m,n in zip(pred_y, real_y):
    if m==n:
        correct +=1
acc = correct / len(real_y)
print(acc) #20.192173791950982%

0.027979801187791788


In [21]:
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR

linearpred = nn.Linear(50, 51) # 使用cos分数预测

optimizer = optim.SGD(linearpred.parameters() ,lr=0.1,
                momentum=0.9,
                dampening=0,
                weight_decay=0.005)

scheduler = MultiStepLR(optimizer, milestones=[20, 20, 20], gamma=0.1)

output_softmax = nn.Softmax(dim=1)

crossentropy = nn.CrossEntropyLoss()

In [22]:
epoch = 80
t_x = torch.tensor(x).cuda()
print(t_x.shape)
linearpred = linearpred.cuda()
t_y = torch.tensor(y).long().cuda()
print(t_y.shape)

for i in range(epoch):
    
    p_y = linearpred(t_x)
    loss = crossentropy(p_y,t_y)
    
    p_y_lab = output_softmax(p_y)
    pred_one_labels = torch.max(p_y_lab, dim=1)[1]
    corrects = float((pred_one_labels.cuda() == t_y).sum().item())
    accuracy = corrects / len(y)
    
    if (i+1) % 5 == 0:
        print("Accuracy is %.4f %% " % (accuracy*100.)) # 100%
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

torch.Size([1392, 50])
torch.Size([1392])
Accuracy is 91.0201 % 
Accuracy is 98.2759 % 
Accuracy is 99.5690 % 
Accuracy is 99.8563 % 
Accuracy is 99.9282 % 
Accuracy is 99.9282 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 
Accuracy is 100.0000 % 


In [23]:
lin_pre_x = torch.tensor(pre_x).cuda(0)
lin_pre_y = linearpred(lin_pre_x)
lin_p_y_lab = output_softmax(lin_pre_y)
lin_pred_labels = torch.max(lin_p_y_lab, dim=1)[1]

lin_real_y = torch.tensor(real_y).long().cuda()

corrects = float((lin_pred_labels.cuda() == lin_real_y).sum().item())
lin_test_accuracy = corrects / len(real_y)
print(lin_test_accuracy*100.) # 26.65%

0.3652300949598247
